## Exercise 2

* Tensorboard report is not getting converted when pdf is generated, I have added the screenshots of some plots but I have also attached the log files folder which can be used to view the plots and images on a tensorboard running instance.

In [4]:
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import torch as pt 
from torch.utils.data import Dataset, DataLoader , Subset, ConcatDataset, ChainDataset
from torch.nn import functional as F
from torch import nn
from torchvision.datasets import MNIST, CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.tensorboard import SummaryWriter


In [5]:
%load_ext tensorboard
# Here also we are working with MNIST dataset so most of the code is reused from 1st exercise.
tr_data = MNIST(root = ".", train = True, download = True, transform = ToTensor())
ts_data = MNIST(root = ".", train = False, download = True, transform = ToTensor())

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
# Here we make few changes to our existing model from previous assignment. As here we add some more 
# linear transformation where in the end we get single output. Here in forward funcion I am using with each layer
# instead of sigmoid because it is fast and avoids vanishing gradient problem. 
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (5, 5), padding=2)
        self.conv2 = nn.Conv2d(6, 16, (5, 5))
        self.fc1   = nn.Linear(16 * 5 * 5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
        self.fc4   = nn.Linear(10, 1)
    
    def forward(self, x):
        N, K, C, W, H = x.size()
        x = x.view(N*K, C, W, H)
        x = F.max_pool2d(pt.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(pt.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, 16 * 5 * 5)
        x = pt.relu(self.fc1(x))
        x = pt.relu(self.fc2(x))
        x = pt.relu(self.fc3(x))
        x = pt.relu(self.fc4(x))
        x = x.view(N, K)
        x = pt.sum(x, dim = 1)
        return x

In [32]:
# defining some hyper-params
lr = 1e-3
b_size = 64
K = 5

In [8]:
# We make minor changes in training and testing funcion like in this problem we dont need to keep track
# of accuracy. However, I am still using the same adam optimizer just to have same scenario for all problems 
def lenet_tr(dataLoad, c_model, loss_func, adam_opt):
    dataSize = len(dataLoad.dataset)
    avg_loss, accu = 0, 0
    for batch, (X, y) in tqdm(enumerate(dataLoad), leave = False):
        X, y = X.to('cpu'), y.to('cpu')
        pred = c_model(X)
        loss_val = loss_func(pred, y)
        adam_opt.zero_grad()
        loss_val.backward()
        adam_opt.step()
        avg_loss = avg_loss + (loss_val.item() * X.size(0))

    avg_loss = avg_loss / (dataSize * K)
    print("Training Average loss: ", str(avg_loss))
    return avg_loss


def lenet_ts(dataLoad, c_model, loss_func):
    dataSize = len(dataLoad.dataset)
    ts_loss, accu = 0, 0
    with pt.no_grad():
        for X, y in tqdm(dataLoad,leave = False):
            X, y = X.to('cpu'), y.to('cpu')
            pred = c_model(X)
            ts_loss = ts_loss + (loss_func(pred, y).item() * X.size(0))

    ts_loss = ts_loss / (dataSize * K)
    print("Test Average loss: ", str(ts_loss))
    return ts_loss

In [9]:
# This is the function which is then responsible to return us K batches of images as mentioned in the question
# Then this custom dataset is being used in training 
class Batch_Shaped_DS(Dataset):
    def __init__(self, K, normal_dataset):
        self.K = K
        self.normal_dataset = normal_dataset
        self.len_original = len(self.normal_dataset)
        self.on_epoch_end()

    def __len__(self):
        return self.len_original//self.K

    def __getitem__(self, idx):
        ind = self.index_list[idx]
        X = [self.normal_dataset.__getitem__(i)[0] for i in ind]
        y = [self.normal_dataset.__getitem__(i)[1] for i in ind]
        return pt.stack(X), np.sum(y).astype(dtype = np.float32)

    def on_epoch_end(self):
        self.index_list = np.random.randint(0, self.len_original, size = (self.len_original // self.K, self.K))

update_tr_data = Batch_Shaped_DS(K = K, normal_dataset = tr_data)
update_ts_data = Batch_Shaped_DS(K = K, normal_dataset = ts_data)

update_tr_dl = DataLoader(update_tr_data, batch_size = b_size, shuffle = True)
update_ts_dl = DataLoader(update_ts_data, batch_size = b_size, shuffle = True)


In [10]:
# This is the main code where we are creating our model and all the epochs are happening in a similar manner as
# done in previous parts. Here I ran for max 30 epochs because it was taking quite time even with relu
# and similarly stored the results using summarywriter.
c_model = LeNet()
c_model.to('cpu')
loss_func = nn.MSELoss()
adam_opt = pt.optim.Adam(c_model.parameters(), lr = lr, betas = (0.9, 0.999), eps = 1e-08, weight_decay = 0, amsgrad = False)
sum_output = SummaryWriter('runs/experiment_2')

max_iter = 30
for i in tqdm(range(max_iter)):
    print("Epoch: " , str(i+1))
    tr_loss = lenet_tr(update_tr_dl, c_model, loss_func, adam_opt)
    update_tr_dl.dataset.on_epoch_end()

    ts_loss = lenet_ts(update_ts_dl, c_model, loss_func)    
    update_ts_dl.dataset.on_epoch_end()

    if sum_output is not None:
        sum_output.add_scalars(f'Loss with lr = {lr}', {"Train" : tr_loss,
                                     "Test" : ts_loss}, i)

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch:  1


0it [00:00, ?it/s]

Training Average loss:  16.826628247070314


  0%|          | 0/32 [00:00<?, ?it/s]

...

Test Average loss:  0.43536392707824706
Epoch:  30


0it [00:00, ?it/s]

Training Average loss:  0.4567667993545532


  0%|          | 0/32 [00:00<?, ?it/s]

Test Average loss:  0.4667197177886963


In [24]:
# This is the part where we perform testin and check the predicted output with actual output.
# We store this analysis again using sumarywriter so that later it can be seen with tensorbaord.
for K in range(10, 11):
    update_ts_data = Batch_Shaped_DS(K = K, normal_dataset = ts_data)
    update_ts_dl = DataLoader(update_ts_data, batch_size = b_size, shuffle = True)

    smpl = iter(update_ts_dl).next()
    out = c_model(smpl[0].to('cpu'))
    smpl[1], out

    sub_smpl = np.random.randint(0, len(smpl[1]), size = (10,))

    fig, axes = plt.subplots(len(sub_smpl), K, figsize = (1.5 * K, 2 * len(sub_smpl)))
    for i, ind in enumerate(sub_smpl):
        temp = smpl[0][ind], smpl[1][ind]
        for j in range(K):
            axes[i, j].imshow(temp[0][j].numpy()[0])
            axes[i, j].axis('off')
        axes[i, j//2].set_title(f'Actual : {temp[1]}\nPredicted : {out[ind]:.02f}',
                               color = ('green' if np.abs(temp[1].cpu().detach().numpy() - out[ind].cpu().detach().numpy()) < 1 else 'red')) 
        sum_output.add_figure(f'Predictions Vs Actual for K : {K}', fig)
    plt.close()

sum_output.close()


In [31]:
%tensorboard --logdir runs/experiment_2 --host localhost --port 8091
